In [52]:
import sys
import os

print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

# Get the absolute path to the directory containing the current script
# For GlobalLocal/src/analysis/preproc/make_epoched_data.py, this is GlobalLocal/src/analysis/preproc
try:
    # This will work if running as a .py script
    current_file_path = os.path.abspath(__file__)
    current_script_dir = os.path.dirname(current_file_path)
except NameError:
    # This will be executed if __file__ is not defined (e.g., in a Jupyter Notebook)
    # os.getcwd() often gives the directory of the notebook,
    # or the directory from which the Jupyter server was started.
    current_script_dir = os.getcwd()

# Navigate up three levels to get to the 'GlobalLocal' directory
project_root = os.path.abspath(os.path.join(current_script_dir, '..'))

# Add the 'GlobalLocal' directory to sys.path if it's not already there
if project_root not in sys.path:
    sys.path.insert(0, project_root) # insert at the beginning to prioritize it

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from bids import BIDSLayout
import mne
import numpy as np
import copy
import seaborn as sns
import pandas as pd
from scipy import stats
import math

import pickle
from src.analysis.utils.general_utils import get_good_data, get_sig_chans_per_subject
from src.analysis.preproc.make_epoched_data import trial_ieeg_rand_offset


['c:\\Users\\jz421\\Desktop\\GlobalLocal', 'c:\\Users\\jz421', 'C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anac

In [60]:
if LAB_root is None:
    HOME = os.path.expanduser("~")
    if os.name == 'nt': # windows
        LAB_root = os.path.join(HOME, "Box", "CoganLab")
    else: # mac
        LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

layout = get_data(task, root=LAB_root)

### actually make the baselines so we can compare them later

In [ ]:
# subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103', 'D0107A', 'D0110', 'D0116', 'D0117', 'D0121']
subjects = ['D0057']
task = 'GlobalLocal'
LAB_root = None
channels = None
pad_length = 0.5
base_times_length = 0.5
outliers = 10
passband = (70,150)
dec_factor = 8

baseline_configs = [
    {
        "name": "Stimulus_100sec_within1-101sec_experimentStartBase",
        "baseline_event": "experimentStart",
        "base_times": (1, 101),
        "base_times_length": 100, # The total duration of the baseline epoch
        "rand_offset": False,
    },

    {
        "name": "Stimulus_0.5sec_within1-101sec_randoffset_experimentStartBase",
        "baseline_event": "experimentStart",
        "base_times": (1, 101),
        "base_times_length": 0.5, # The total duration of the baseline epoch
        "rand_offset": True,
    },

    # {
    #     "name": "Stimulus_0.5sec_within1sec_fixed_preStimulusBase",
    #     "baseline_event": "Stimulus",
    #     "base_times": (-0.5, 0),
    #     "base_times_length": 0.5, # The duration of the snippet taken from the window
    #     "rand_offset": False,
    # },

    {
        "name": "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase",
        "baseline_event": "Stimulus",
        "base_times": (-0.5, 0),
        "base_times_length": 0.5, # The duration of the snippet taken from the window
        "rand_offset": True,
    },

    # {
    #     "name": "Stimulus_1.0sec_within1sec_randoffset_preStimulusBase",
    #     "baseline_event": "Stimulus",
    #     "base_times": (-1, 0),
    #     "base_times_length": 1.0,
    #     "rand_offset": False,
    # },

    # {
    #     "name": "Stimulus_0.2sec_within1sec_fixed_preStimulusBase",
    #     "baseline_event": "Stimulus",
    #     "base_times": (-0.2, 0),
    #     "base_times_length": 0.2,
    #     "rand_offset": False,
    # },

    # {
    #     "name": "Stimulus_0.2sec_within1sec_randoffset_preStimulusBase",
    #     "baseline_event": "Stimulus",
    #     "base_times": (-0.2, 0),
    #     "base_times_length": 0.2,
    #     "rand_offset": True,
    # }
]

In [79]:
def get_baseline(inst: mne.io.BaseRaw, baseline_event: str, base_times: tuple[float, float], pad_length: float, base_times_length: float, outliers: int, passband: tuple[float, float], dec_factor: int, rand_offset: bool):
    inst = inst.copy()
    inst.load_data()
    ch_type = inst.get_channel_types(only_data_chs=True)[0]
    inst.set_eeg_reference(ref_channels="average", ch_type=ch_type)

    if rand_offset:
        trials = trial_ieeg_rand_offset(inst, baseline_event, base_times, base_times_length, pad_length, preload=True)
    else:
        adjusted_base_times = [base_times[0] - pad_length, base_times[1] + pad_length]
        trials = trial_ieeg(inst, baseline_event, adjusted_base_times, preload=True)

    outliers_to_nan(trials, outliers=outliers)
    HG_base = gamma.extract(trials, passband=passband, copy=False, n_jobs=1)
    crop_pad(HG_base, f"{pad_length}s")
    HG_base.decimate(dec_factor)

    del inst
    return HG_base

In [100]:
def make_baselines(subjects, task, baseline_configs, LAB_root, pad_length, outliers, passband, dec_factor):
    """
    Creates baselines by iterating through a list of configuration dictionaries.
    
    Parameters:
    subjects: list of subjects to create baselines for
    task: task to create baselines for
    baseline_configs: list of configuration dictionaries for baselines
    LAB_root: root directory of the lab data
    pad_length: length of padding to use for baseline creation
    outliers: number of outliers to use for baseline creation
    passband: passband to use for baseline creation
    dec_factor: decimation factor to use for baseline creation
    """
    if LAB_root is None:
        HOME = os.path.expanduser("~")
        LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

    layout = get_data(task, root=LAB_root)

    for sub in subjects:
        good = get_good_data(sub, layout)

        # Loop through the list of configurations
        for config in baseline_configs:
            print(f"--- Creating baseline: {config['name']} for subject {sub} ---")
            
            # Pass parameters from the config dictionary
            HG_base = get_baseline(good, 
                                   baseline_event=config['baseline_event'], 
                                   base_times=config['base_times'],
                                   base_times_length=config['base_times_length'],
                                   rand_offset=config['rand_offset'],
                                   pad_length=pad_length, 
                                   outliers=outliers, 
                                   passband=passband, 
                                   dec_factor=dec_factor)

            # Use the unique config name for the save directory
            save_dir = os.path.join(layout.root, 'derivatives', 'tests', 'baselineTesting', config['name'], sub)
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)

            output_name_base = generate_output_name(config, dec_factor, outliers, passband, pad_length)
            
            HG_base.save(os.path.join(save_dir, f"{sub}_{output_name_base}_base-epo.fif"), overwrite=True)

In [97]:
def generate_output_name(config, dec_factor, outliers, passband, pad_length):
    """Generates a standardized, descriptive name from a config dictionary."""
    
    # Use the specific length from the config
    name = config['name']
    
    offset_str = "randoffset_" if config['rand_offset'] else ""
    
    return (f"{name}_decFactor_{dec_factor}_"
            f"outliers_{outliers}_passband_{passband[0]}-{passband[1]}_"
            f"padLength_{pad_length}s")

In [101]:
# --- Call the new function ---
make_baselines(subjects, task, baseline_configs, LAB_root, pad_length, outliers, passband, dec_factor)

Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_ieeg.edf...
EDF file detected
Setting channel info str

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Omitted 228 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-04_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


...

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Omitted 226 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


<RawEDF | sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_ieeg.edf, 178 x 8243200 (4025.0 s), ~201 kB, data not loaded>
outlier round 1 channels: ['RAMT8']
outlier round 2 channels: ['RAMT8', 'RPI16']
outlier round 2 channels: ['RAMT8', 'RPI16', 'LAMT12']
Reading 0 ... 6198083  =      0.000 ...  3026.408 secs...
Applying average reference.
Applying a custom ('sEEG',) reference.
--- Creating baseline: Stimulus_100sec_within1-101sec_experimentStartBase for subject D0057 ---
Applying average reference.
Applying a custom ('sEEG',) reference.
Used Annotations descriptions: ['1', '2', 'Response/c25.0/n25.0/BigLetters/SmallLetters/Taskl/TargetLetters/Responded1.0/ParticipantResponse115.0/CorrectResponse115.0/TrialCount225.0/BlockTrialCount1.0/ReactionTime2149.0/Accuracy1.0/D57', 'Response/c25.0/r25.0/BigLetterh/SmallLetterh/Taskg/TargetLetterh/Responded1.0/ParticipantResponse114.0/CorrectResponse114.0/TrialCount227.0/BlockTrialCount3.0/ReactionTime883.0/Accuracy1.0/D57', 'Response/c25.0/r

100%|██████████| 1/1 [00:30<00:00, 30.30s/it]
C:\Users\jz421\AppData\Local\Temp\ipykernel_16376\2499320382.py:16: RuntimeWarning: The measurement information indicates a low-pass frequency of 1024.0 Hz. The decim=8 parameter will result in a sampling frequency of 256.0 Hz, which can cause aliasing artifacts.
  HG_base.decimate(dec_factor)


--- Creating baseline: Stimulus_0.5sec_within1sec_randoffset_preStimulusBase for subject D0057 ---
Applying average reference.
Applying a custom ('sEEG',) reference.
Used Annotations descriptions: ['1', '2', 'Response/c25.0/n25.0/BigLetters/SmallLetters/Taskl/TargetLetters/Responded1.0/ParticipantResponse115.0/CorrectResponse115.0/TrialCount225.0/BlockTrialCount1.0/ReactionTime2149.0/Accuracy1.0/D57', 'Response/c25.0/r25.0/BigLetterh/SmallLetterh/Taskg/TargetLetterh/Responded1.0/ParticipantResponse114.0/CorrectResponse114.0/TrialCount227.0/BlockTrialCount3.0/ReactionTime883.0/Accuracy1.0/D57', 'Response/c25.0/r25.0/BigLetterh/SmallLetterh/Taskg/TargetLetterh/Responded1.0/ParticipantResponse114.0/CorrectResponse114.0/TrialCount228.0/BlockTrialCount4.0/ReactionTime766.0/Accuracy1.0/D57', 'Response/c25.0/r25.0/BigLetterh/SmallLetterh/Taskg/TargetLetterh/Responded1.0/ParticipantResponse114.0/CorrectResponse114.0/TrialCount241.0/BlockTrialCount17.0/ReactionTime499.0/Accuracy1.0/D57', 'Respo

100%|██████████| 449/449 [02:07<00:00,  3.52it/s]
C:\Users\jz421\AppData\Local\Temp\ipykernel_16376\2499320382.py:16: RuntimeWarning: The measurement information indicates a low-pass frequency of 1024.0 Hz. The decim=8 parameter will result in a sampling frequency of 256.0 Hz, which can cause aliasing artifacts.
  HG_base.decimate(dec_factor)


load in some baselines for testing, and also load in the signal to compare them to

In [102]:
loaded_baselines = {}
loaded_signals = {}

for sub in subjects:
    signal_path = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub, f'{sub}_Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False_HG_ev1-epo.fif')
    loaded_signals[sub] = mne.read_epochs(signal_path)
    loaded_baselines[sub] = {}

    # Loop through the same configs to load the data you just created
    for config in baseline_configs:
        # Reconstruct the path with our consistent naming scheme
        save_dir = os.path.join(layout.root, 'derivatives', 'tests', 'baselineTesting', config['name'], sub)
        output_name_base = generate_output_name(config, dec_factor, outliers, passband, pad_length)
        file_path = os.path.join(save_dir, f"{sub}_{output_name_base}_base-epo.fif")

        if os.path.exists(file_path):
            # Use the unique config name as the key
            loaded_baselines[sub][config['name']] = mne.read_epochs(file_path)
        else:
            print(f"Warning: File not found for {sub}, baseline {config['name']}: {file_path}")

Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False_HG_ev1-epo.fif ...


    Found the data of interest:
        t =   -1000.00 ...    1500.00 ms
        0 CTF compensation matrices available
Not setting metadata
449 matching events found
No baseline correction applied
0 projection items activated
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\tests\baselineTesting\Stimulus_100sec_within1-101sec_experimentStartBase\D0057\D0057_Stimulus_100sec_within1-101sec_experimentStartBase_decFactor_8_outliers_10_passband_70-150_padLength_0.5s_base-epo.fif ...
    Found the data of interest:
        t =    1000.00 ...  101000.00 ms
        0 CTF compensation matrices available
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\tests\baselineTesting\Stimulus_0.5sec_within1sec_randoffset_preStimulusBase\D0057\D0057_Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70-150_padLength_0.

grab sig elecs

In [103]:
# Now, the keys here should match the short, unique names from your configs.
# This makes the lookup direct and removes the need for fuzzy matching.
sig_chans_per_baseline = {}

for config in baseline_configs:
    # --- Generate all potential filenames first ---
    
    # Create a name with integer-formatted passband
    passband_as_int = (int(passband[0]), int(passband[1]))
    sig_run_name_int = generate_output_name(config, dec_factor, outliers, passband_as_int, pad_length) + "_stat_func_ttest_ind_equal_var_False"
    
    # Create a name with float-formatted passband
    passband_as_float = (float(passband[0]), float(passband[1]))
    sig_run_name_float = generate_output_name(config, dec_factor, outliers, passband_as_float, pad_length) + "_stat_func_ttest_ind_equal_var_False"
    
    # Create a list of unique potential names to try
    potential_names = [sig_run_name_int]
    if sig_run_name_int != sig_run_name_float:
        potential_names.append(sig_run_name_float)

    # --- Attempt to load the data using the potential names ---
    loaded_successfully = False
    for i, sig_run_name in enumerate(potential_names):
        try:
            print(f"Attempting ({i+1}/{len(potential_names)}) to load sig data for '{config['name']}' with name: '{sig_run_name}'")
            sig_chans_per_baseline[config['name']] = get_sig_chans_per_subject(subjects, sig_run_name, task='GlobalLocal', LAB_root=LAB_root)
            print("... Success.")
            loaded_successfully = True
            break # Exit the loop as soon as we find a valid file
            
        except (FileNotFoundError, IOError) as e:
            print(f"... Failed to find file. Error: {e}")
        except Exception as e:
            # Catch any other unexpected errors from get_sig_chans_per_subject
            print(f"... An unexpected error occurred: {e}")

    # If all attempts failed, ensure a key exists with an empty value
    if not loaded_successfully:
        print(f"Warning: Could not load any significance data for config: '{config['name']}'. Assigning empty result.")
        sig_chans_per_baseline[config['name']] = {}

Attempting (1/2) to load sig data for 'Stimulus_100sec_within1-101sec_experimentStartBase' with name: 'Stimulus_100sec_within1-101sec_experimentStartBase_decFactor_8_outliers_10_passband_70-150_padLength_0.5s_stat_func_ttest_ind_equal_var_False'
Loaded significant channels for subject D0057
... Success.
Attempting (1/2) to load sig data for 'Stimulus_0.5sec_within1sec_randoffset_preStimulusBase' with name: 'Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70-150_padLength_0.5s_stat_func_ttest_ind_equal_var_False'
... Failed to find file. Error: [Errno 2] No such file or directory: 'C:\\Users\\jz421\\Box\\CoganLab\\BIDS-1.1_GlobalLocal\\BIDS\\derivatives\\freqFilt\\figs\\D0057\\sig_chans_D0057_Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70-150_padLength_0.5s_stat_func_ttest_ind_equal_var_False.json'
Attempting (2/2) to load sig data for 'Stimulus_0.5sec_within1sec_randoffset_preStimulusBase' with name: 'Sti

now let's play around with the loaded data

all baselines vs signal

In [109]:
import os
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches # ### FIX: Import mpatches to create legend handles
from scipy.stats import gaussian_kde

# Assume the following variables are pre-loaded and defined:
# subjects, loaded_signals, layout, baseline_configs, loaded_baselines, sig_chans_per_baseline

# --- Main Loop ---
# Loop through each subject
for sub in subjects:
    signal = loaded_signals[sub]
    print(f"--- Processing Subject: {sub} ---")

    # Define a single save directory for the multi-baseline comparison plots
    save_dir = os.path.join(layout.root, 'derivatives', 'tests', 'baselineTesting', 'figs', sub)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # First, find the set of channels that are common to the signal AND ALL baseline files
    common_channels = set(signal.ch_names)
    for config in baseline_configs: # Use the new config structure
        baseline_name = config['name']
        if sub in loaded_baselines and baseline_name in loaded_baselines[sub]:
            base = loaded_baselines[sub][baseline_name]
            common_channels.intersection_update(base.ch_names)
    
    # Sort the channels for consistent ordering
    common_channels = sorted(list(common_channels))
    
    print(f"Found {len(common_channels)} channels common to signal and all baselines. Generating plots...")

    # --- Grid Plotting Setup ---
    n_channels = len(common_channels)
    if n_channels == 0:
        print("No common channels found across all conditions, skipping subject.")
        continue
        
    rows = 6
    cols = 10
    plots_per_fig = rows * cols
    n_figs = math.ceil(n_channels / plots_per_fig)

    # ### FIX: Create a unified color map for all data types (Signal + Baselines)
    # This ensures a consistent color for each condition across all plot types (KDE, vline, asterisk).
    all_types = ['Signal (0-1.5s)'] + [c['name'] for c in baseline_configs]
    palette_base = ["#E41A1C"] + sns.color_palette("viridis_r", len(baseline_configs))
    color_map = {name: color for name, color in zip(all_types, palette_base)}

    # --- Figure Generation Loop ---
    for fig_num in range(n_figs):
        fig, axes = plt.subplots(rows, cols, figsize=(cols * 5, rows * 4.5))
        
        start_idx = fig_num * plots_per_fig
        end_idx = min(start_idx + plots_per_fig, n_channels)
        channels_for_fig = common_channels[start_idx:end_idx]

        # --- Channel-level Processing and Plotting ---
        for i, ch_name in enumerate(channels_for_fig):
            ax = axes.flat[i]

            # --- 1. Aggregate Time-Averaged Data ---
            dfs_to_concat = []
            single_trial_baseline_data = {}
            asterisks_to_plot = []

            # a) Get signal data
            signal_cropped = signal.copy().crop(tmin=0, tmax=1.5)
            signal_data = signal_cropped.get_data(picks=ch_name).mean(axis=2).flatten()
            signal_data = signal_data[~np.isnan(signal_data)]
            if signal_data.size > 0:
                dfs_to_concat.append(pd.DataFrame({'Activity': signal_data, 'Type': 'Signal (0-1.5s)'}))

            # b) Get data for each baseline
            for j, config in enumerate(baseline_configs):
                baseline_name = config['name']
                
                if baseline_name not in loaded_baselines.get(sub, {}):
                    print(f'Baseline {baseline_name} not found for subject {sub}. Skipping...')
                    continue

                base = loaded_baselines[sub][baseline_name]
                base_cropped = base.copy().crop(tmin=config['base_times'][0], tmax=config['base_times'][1])
                base_data = base_cropped.get_data(picks=ch_name).mean(axis=2).flatten()
                base_data = base_data[~np.isnan(base_data)]
                
                if ch_name in sig_chans_per_baseline.get(baseline_name, {}).get(sub, []):
                    # ### FIX: Store baseline name instead of color for later lookup in color_map
                    asterisks_to_plot.append({'data': base_data, 'name': baseline_name})

                if base_data.size == 1:
                    single_trial_baseline_data[baseline_name] = base_data[0]
                elif base_data.size > 1:
                    dfs_to_concat.append(pd.DataFrame({'Activity': base_data, 'Type': baseline_name}))

            if not dfs_to_concat and not single_trial_baseline_data:
                ax.set_title(f'{ch_name}\n(No data)', fontsize=8)
                ax.axis('off')
                continue

            df_combined = pd.concat(dfs_to_concat) if dfs_to_concat else pd.DataFrame({'Activity': [], 'Type': []})

            # --- 2. Visualize the Distributions ---
            if not df_combined.empty:
                sns.kdeplot(data=df_combined, x='Activity', hue='Type', 
                            palette=color_map, # ### FIX: Use the unified color_map
                            fill=True, common_norm=False, ax=ax, legend=False)
            
            # --- 3. Plot Single-Trial Baselines as Vertical Lines ---
            for label, value in single_trial_baseline_data.items():
                ax.axvline(x=value, 
                           color=color_map.get(label, 'k'), # ### FIX: Get color from unified color_map
                           linestyle='--', linewidth=2, label=label)

            # --- 4. Plot Asterisks for Significant Channels ---
            if asterisks_to_plot:
                ylim = ax.get_ylim()
                ax.set_ylim(top=ylim[1] * 1.15) # Add extra space for asterisks
                ylim = ax.get_ylim()
                
                # ### FIX: Stagger asterisks vertically to prevent overlap
                y_offset_factor = 0.08 
                
                for k, ast_info in enumerate(asterisks_to_plot):
                    peak_y = ylim[1] * (0.95 - k * y_offset_factor)
                    color = color_map.get(ast_info['name'], 'k') # ### FIX: Get color from map
                    
                    if len(ast_info['data']) == 1:
                        peak_x = ast_info['data'][0]
                    elif len(ast_info['data']) > 1:
                        kde = gaussian_kde(ast_info['data'])
                        x_range = np.linspace(min(ast_info['data']), max(ast_info['data']), 500)
                        kde_vals = kde(x_range)
                        peak_x = x_range[np.argmax(kde_vals)]
                    
                    ax.text(peak_x, peak_y, '*', color=color, 
                            fontsize=24, ha='center', va='center', weight='bold')

            # --- 5. Annotate and Finalize Subplot ---
            ax.set_title(f'Channel: {ch_name}', fontsize=10)
            ax.set_xlabel('Time-Averaged High Gamma Activity', fontsize=8)
            ax.set_ylabel('Density', fontsize=8)
            ax.tick_params(axis='x', labelsize=7, rotation=30)
            ax.tick_params(axis='y', labelsize=7)

            # ### FIX: Create a single, unified legend on the first subplot ###
            if i == 0:
                handles = []
                # Get handles for vertical lines that were actually plotted
                line_handles, line_labels = ax.get_legend_handles_labels()
                handles.extend(line_handles)

                # Create patch handles for KDE plots that were actually plotted
                plotted_kde_types = df_combined['Type'].unique()
                patch_handles = [mpatches.Patch(color=color_map[name], label=name) 
                                 for name in all_types if name in plotted_kde_types]
                
                handles.extend(patch_handles)
                
                # Create a unique list of handles and labels
                # (dict.fromkeys preserves order and removes duplicates)
                unique_labels_map = {h.get_label(): h for h in handles}
                ax.legend(handles=unique_labels_map.values(), labels=unique_labels_map.keys())


        # --- Figure-level Cleanup and Saving ---
        for i in range(len(channels_for_fig), plots_per_fig):
            axes.flat[i].axis('off')

        fig.suptitle(f'Signal vs. All Baselines (Time-Averaged) for Subject: {sub} (Page {fig_num + 1}/{n_figs})', fontsize=20)
        fig.tight_layout(rect=[0, 0.03, 1, 0.96])
        
        save_name = f"{sub}_multi_baseline_vs_post_stim_signal_comparison_page_{fig_num + 1}.png"
        save_path = os.path.join(save_dir, save_name)
        plt.savefig(save_path, dpi=300)
        print(f"Saved figure to: {save_path}")
        plt.close(fig)

--- Processing Subject: D0057 ---
Found 175 channels common to signal and all baselines. Generating plots...
Saved figure to: C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\tests\baselineTesting\figs\D0057\D0057_multi_baseline_vs_post_stim_signal_comparison_page_1.png
Saved figure to: C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\tests\baselineTesting\figs\D0057\D0057_multi_baseline_vs_post_stim_signal_comparison_page_2.png
Saved figure to: C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\tests\baselineTesting\figs\D0057\D0057_multi_baseline_vs_post_stim_signal_comparison_page_3.png


make time series plots with mean and stdev across trials for different baselines

In [ ]:
# =======================================================================================
# --- REVISED SECTION: Individual Time-Series Comparison Plotting ---
# This section generates separate sets of plots for the signal and each baseline
# condition. For each condition, it plots the mean high-gamma trace over time for
# all common channels across multiple pages.
# =======================================================================================

print("\n--- Starting Individual Time-Series Plotting ---")

# Loop through each subject
for sub in subjects:
    print(f"--- Processing Subject for Individual Time-Series Plots: {sub} ---")

    # --- Create a list of all data objects to plot ---
    # Each item in the list will be a tuple: (string_name, MNE_Epochs_object)
    conditions_to_plot = []

    # 1. Add the Signal data
    conditions_to_plot.append(("Signal", loaded_signals[sub]))

    # 2. Add all available Baseline data objects
    for config in baseline_configs:
        baseline_name = config['name']
        if sub in loaded_baselines and baseline_name in loaded_baselines[sub]:
            conditions_to_plot.append((baseline_name, loaded_baselines[sub][baseline_name]))

    # --- Iterate through each condition (Signal, then each Baseline) ---
    for condition_name, data_object in conditions_to_plot:

        print(f"  -- Generating plots for condition: {condition_name} --")

        # Define a save directory for this specific condition's plots
        save_dir = os.path.join(layout.root, 'derivatives', 'tests', 'baselineTesting', 'figs', sub)
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

        # Determine channels to plot based on the CURRENT data object
        channels_for_this_condition = sorted(data_object.ch_names)

        if not channels_for_this_condition:
            print(f"  -- No channels found for condition {condition_name}, skipping.")
            continue

        # --- Grid Plotting Setup (based on current channels) ---
        rows = 6
        cols = 10
        plots_per_fig = rows * cols
        n_figs = math.ceil(len(channels_for_this_condition) / plots_per_fig)

        # --- Figure Generation Loop (for pagination) ---
        for fig_num in range(n_figs):
            fig, axes = plt.subplots(rows, cols, figsize=(cols * 5, rows * 4.5), constrained_layout=True)

            start_idx = fig_num * plots_per_fig
            end_idx = min(start_idx + plots_per_fig, len(channels_for_this_condition))
            channels_for_fig = channels_for_this_condition[start_idx:end_idx]

            # --- Channel-level Processing and Plotting ---
            for i, ch_name in enumerate(channels_for_fig):
                ax = axes.flat[i]

                # Get the data for the current channel
                timeseries_data = data_object.get_data(picks=ch_name, copy=True)
                mean_trace = np.nanmean(timeseries_data, axis=0)
                std_trace = np.nanstd(timeseries_data, axis=0)

                if i == 1:
                    print(std_trace)

                time_vector = data_object.times

                # Plot the main trace and standard deviation
                ax.plot(time_vector, mean_trace, color="#2c7fb8", linewidth=1.5)
                ax.fill_between(time_vector, mean_trace - std_trace, mean_trace + std_trace, color="#2c7fb8", alpha=0.3)

                # --- MODIFICATION: Add baseline info ONLY to Signal plots ---
                plot_title = f'Channel: {ch_name}' # Default title

                if condition_name == "Signal":
                    # 1. Add black asterisk to TITLE for significance vs. PRE-STIMULUS baseline
                    prestim_key = 'Stimulus_0.5sec_within1sec_randoffset_preStimulusBase'
                    sig_chans_for_sub_prestim = sig_chans_per_baseline.get(prestim_key, {}).get(sub, [])
                    if ch_name in sig_chans_for_sub_prestim:
                        plot_title += ' *'

                    # 2. Add horizontal bar for EXPERIMENT START baseline mean
                    exp_start_key = 'Stimulus_100sec_within1-101sec_experimentStartBase'
                    if sub in loaded_baselines and exp_start_key in loaded_baselines[sub]:
                        base_obj = loaded_baselines[sub][exp_start_key]
                        base_data = base_obj.get_data(picks=ch_name, copy=True)
                        mean_base_activity = np.nanmean(base_data)

                        # Always plot the horizontal line itself
                        ax.axhline(y=mean_base_activity, color='r', linestyle='--', linewidth=2.5)

                        # --- FIX: Only plot the red asterisk if the channel is significant vs. THIS baseline ---
                        sig_chans_for_sub_exp_start = sig_chans_per_baseline.get(exp_start_key, {}).get(sub, [])
                        if ch_name in sig_chans_for_sub_exp_start:
                            # This asterisk only appears if the signal is significantly different from the experiment start baseline
                            ax.text(time_vector[-1], mean_base_activity, '*', color='r', fontsize=24,
                                    ha='center', va='center', weight='bold')

                # --- Finalize Subplot ---
                ax.set_title(plot_title, fontsize=10) # Use the potentially modified title
                ax.set_xlabel('Time (s)', fontsize=8)
                ax.set_ylabel('High Gamma', fontsize=8)
                ax.tick_params(axis='both', labelsize=7)
                ax.axvline(0, color='k', linestyle=':', linewidth=1.5)
                ax.grid(True, which='both', linestyle='--', linewidth=0.5)
                ax.set_xlim(time_vector[0], time_vector[-1])

            # --- Figure-level Cleanup and Saving ---
            for i in range(len(channels_for_fig), plots_per_fig):
                axes.flat[i].axis('off')

            fig.suptitle(f'Time-Series for {condition_name} - Subject: {sub} (Page {fig_num + 1}/{n_figs})', fontsize=20)

            safe_condition_name = condition_name.replace("/", "_")
            save_name = f"{sub}_{safe_condition_name}_timeseries_page_{fig_num + 1}.png"
            save_path = os.path.join(save_dir, save_name)

            plt.savefig(save_path, dpi=300)
            print(f"    Saved figure to: {save_path}")
            plt.close(fig)


--- Starting Individual Time-Series Plotting ---
--- Processing Subject for Individual Time-Series Plots: D0057 ---
  -- Generating plots for condition: Signal --
Backend Qt5Agg is interactive backend. Turning interactive mode on.
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

C:\Users\jz421\AppData\Local\Temp\ipykernel_16376\4266245769.py:91: RuntimeWarning: Mean of empty slice
  mean_base_activity = np.nanmean(base_data)


KeyboardInterrupt: 

### average across trials and timepoints but not channels, plot avg mean and stdev for each channel

In [20]:
data = HG_base_stimulus.get_data()

mean = np.mean(data, axis=(0,2))
stdev = np.std(data,axis=(0,2))

print("Mean: ", mean)
print("Standard Deviation: ", stdev)

# Extract the channel names from the Epochs object
channel_names = HG_base_stimulus.ch_names

# Create a bar plot
plt.figure(figsize=(10, 5))
plt.bar(channel_names, mean, yerr=stdev, align='center', alpha=0.75, ecolor='black', capsize=5)

# Customize the plot
plt.ylabel('Mean Amplitude')
plt.title('Mean and Standard Deviation of Channels')
plt.xticks(rotation=90)  # Rotate the channel names for better readability

# Display the plot
plt.tight_layout()
plt.show()

Mean:  [2.35596596e-05            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
 7.60397970e-06 7.19475317e-06 6.55293008e-06 6.04449749e-06
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan  

KeyboardInterrupt: 

### average across trials, channels, and time points

In [21]:
data = HG_base.get_data()

mean = np.nanmean(data, axis=(0,1,2))
stdev = np.nanstd(data,axis=(0,1,2))

print("Mean of experiment start baseline: ", mean)
print("Standard Deviation of experiment start baseline: ", stdev)

Mean of experiment start baseline:  7.11109295788856e-06
Standard Deviation of experiment start baseline:  2.467975522575185e-06


In [22]:
data = HG_base_stimulus.get_data()

mean = np.nanmean(data, axis=(0,1,2))
stdev = np.nanstd(data,axis=(0,1,2))

print("Mean of fixation cross baseline: ", mean)
print("Standard Deviation of fixation cross baseline: ", stdev)

Mean of fixation cross baseline:  6.0587577502376924e-06
Standard Deviation of fixation cross baseline:  4.567779012040867e-06


### average across trials and channels but not timepoints

In [23]:
data = HG_base.get_data()

mean = np.nanmean(data, axis=(0,1))
stdev = np.nanstd(data,axis=(0,1))

print("Mean: ", mean)
print("Standard Deviation: ", stdev)

# Extract the channel names from the Epochs object
channel_names = HG_base.ch_names


# Now, create a plot
plt.figure(figsize=(10, 5))
plt.plot(HG_base.times, mean, label='Mean')
plt.fill_between(HG_base.times, mean - stdev, mean + stdev, alpha=0.2, label='STD')

# Customize the plot
plt.ylabel('Mean Amplitude')
plt.xlabel('Time (s)')
plt.title('Mean and Standard Deviation of timepoints across trials and channels')
plt.legend(loc='upper right')

# Display the plot
plt.tight_layout()
plt.show()

Mean:  [6.21817376e-06 6.23033093e-06 6.24244690e-06 ... 7.53210802e-06
 7.54389375e-06 7.55552610e-06]
Standard Deviation:  [1.16503499e-06 1.17829252e-06 1.19179487e-06 ... 2.64761902e-06
 2.64209893e-06 2.63672973e-06]


KeyboardInterrupt: 

In [24]:
data = HG_base_stimulus.get_data()

mean = np.nanmean(data, axis=(0,1))
stdev = np.nanstd(data,axis=(0,1))

print("Mean: ", mean)
print("Standard Deviation: ", stdev)

# Extract the channel names from the Epochs object
channel_names = HG_base_stimulus.ch_names


# Now, create a plot
plt.figure(figsize=(10, 5))
plt.plot(HG_base_stimulus.times, mean, label='Mean')
plt.fill_between(HG_base_stimulus.times, mean - stdev, mean + stdev, alpha=0.2, label='STD')

# Customize the plot
plt.ylabel('Mean Amplitude')
plt.xlabel('Time (s)')
plt.title('Mean and Standard Deviation of timepoints across trials and channels')
plt.legend(loc='upper right')

# Display the plot
plt.tight_layout()
plt.show()

Mean:  [5.96941022e-06 5.96859328e-06 5.96779262e-06 ... 6.24856718e-06
 6.24944321e-06 6.25032414e-06]
Standard Deviation:  [4.04477745e-06 4.03667350e-06 4.02874892e-06 ... 4.98159151e-06
 4.97979517e-06 4.97793714e-06]


KeyboardInterrupt: 